# 200726 Intra-taxon distance lower bounds

Let $ \pmb{A} = \{ A_i \}_{i \in 1..n} $ be all the signatures in species A and $ \pmb{B} = \{ B_j \}_{j \in 1..m} $ be all signatures in species B.

Pre-calculate:
$$
\bigcup \pmb{A} \\
\bigcup \pmb{B} \\
a = \min_i \left| A_i \right| \\
b = \min_j \left| B_j \right|
$$

For all species. Then $ \forall i \forall j$:

$$
A_i \cap B_j \subseteq \left(\bigcup \pmb{A}\right) \cap \left(\bigcup \pmb{B}\right) \\
\Rightarrow \left| A_i \cap B_j \right| \leq \left| \left(\bigcup \pmb{A}\right) \cap \left(\bigcup \pmb{B}\right) \right| \\
$$

and

$$ \left| A_i \cup A_j \right| \;\geq\; \max(\left| A_i \right|, \left| B_j \right|) \;\geq\; \max(a, b) $$

so

$$
\frac{\left| A_i \cap A_j \right|}{\left| A_i \cup A_j \right|}
\;\leq\;
\frac{\left| \left(\bigcup \pmb{A}\right) \cap \left(\bigcup \pmb{B}\right) \right|}{\max(a, b)}
$$

gives us an upper bound on the similarity between any two genomes in species A and B without having to look at all pairwise combinations.

In [25]:
using GZip
using ProgressMeter
using DataFrames
using CSV
using StatsBase
using CategoricalArrays
using JLD
using Serialization
using JSON

In [2]:
using Midas
using Midas.Distances
using Midas.SignatureFiles
using TriMatrices

## Func defs

In [3]:
findclass(a::CategoricalArray, cls::CategoricalValue) = findall(==(cls), a)
findclass(a::CategoricalArray, i::Integer) = findclass(a, a.pool[i])
selectclass(a::AbstractVector, c::CategoricalArray, cls) = a[findclass(c, cls)]

selectclass (generic function with 1 method)

## File paths

In [4]:
taxonomy_file = "/Users/student/notebooks/midas/midas-notebooks-2019/build-v1-database/out/3-curated-taxonomy-assignments.csv"
signature_file_name = "/Users/student/projects/midas/data/2019_20/refseq_curated_1.1beta_200604.midas-signatures.gz"
;

In [5]:
tmpdir = "tmp/"
!isdir(tmpdir) && mkdir(tmpdir);

In [86]:
outdir = "../../data/processed/200722-detect-overlaps/"

outfiles = Dict(
    :species_min_dists => joinpath(outdir, "200726-species-min-dists.jld"),
    :genus_min_dists => joinpath(outdir, "200726-genus-min-dists.jld"),
)

Dict{Symbol,String} with 2 entries:
  :species_min_dists => "../../data/processed/200722-detect-overlaps/200726-spe…
  :genus_min_dists   => "../../data/processed/200722-detect-overlaps/200726-gen…

## Load taxonomy

In [6]:
taxdf = DataFrame(CSV.File(taxonomy_file));

In [7]:
sig_genera = categorical(taxdf[!, :genus])
genera = levels(sig_genera)
ngenera = length(genera)

sig_species = categorical([(row[:genus], row[:species]) for row in eachrow(taxdf)])
species = levels(sig_species)
nspecies = length(species)

ngenera, nspecies

(419, 1438)

In [17]:
genus_counts = counts(sig_genera.refs)
species_counts = counts(sig_species.refs)
;

In [9]:
genome_accs = [last(split(k, "/")) for k in taxdf[!, :key]];

In [81]:
species_to_genus = [findfirst(==(genus), genera) for (genus, spname) in species];

## Load signatures

In [10]:
sigfile = SignatureFile(GZip.open(signature_file_name))

SignatureFile{UInt32,GZipStream} with 50752 elements

In [26]:
metadata = SignatureFiles.read_metadata(sigfile)
JSON.print(metadata, 2)

{
  "date_created": "2020-06-04",
  "genome_set": {
    "key": "midas/assembly/curated",
    "name": "refseq_curated_2020",
    "meta": {
      "date_created": "2020-05-26",
      "parent": {
        "key": "midas/assembly/curated",
        "key_version": "0.9"
      }
    },
    "description": "Created 2020-05-26 by filtering version 0.9 by inclusion in refseq/assembly/all 1.1",
    "key_version": "1.1"
  },
  "kmer_spec": {
    "k": 11,
    "prefix": "ATGAC"
  },
  "description": "Signatures for version 1.1 of curated genome set"
}


In [12]:
# Should both be sorted:
@assert sigfile.ids == taxdf[:, :key]

In [20]:
@time sigs = SignatureArray(sigfile);

 97.200928 seconds (5 allocations: 1.352 GiB, 0.12% gc time)


In [21]:
nsigs = length(sigs)

50752

In [22]:
nkmers = 4^11

4194304

## Inter-species distance lower bounds

Minimum number of kmers for genome in each species:

In [44]:
species_minkmers = [minimum(length(sigs[i]) for i in findclass(sig_species, si)) for si in 1:nspecies];

Find union of all kmers in all signatures of each species:

In [23]:
species_kmer_unions = let
    tmpfile = joinpath(tmpdir, "species-kmer-unions.jls")
    
    if isfile(tmpfile)
        deserialize(tmpfile)
        
    else
        kmers_bufs = [falses(nkmers) for _ in 1:Threads.nthreads()];

        progress_pmap(1:nspecies) do si
            buf = kmers_bufs[Threads.threadid()]
            buf[:] .= 0

            for i in findclass(sig_species, si)
                for k in sigs[i]
                    @inbounds buf[k + 1] = true
                end
            end

            findall(buf)
        end
    end
end;

Progress: 100%|█████████████████████████████████████████| Time: 0:01:46


Calculate distance lower bounds:

In [ ]:
species_pw_mindists = zeros(Float32, TriSymmetric(), nspecies);

In [46]:
@showprogress for i in 1:nspecies
    kmers_i = species_kmer_unions[i]
    minsize_i = species_minkmers[i]
    for j in 1:(i-1)
        kmers_j = species_kmer_unions[j]
        minsize_j = species_minkmers[j]
        isize = length(intersect(kmers_i, kmers_j))
        species_pw_mindists[i, j] = 1 - isize / max(minsize_i, minsize_j)
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:38:31


In [87]:
save(outfiles[:species_min_dists], Dict(
    "data" => species_pw_mindists.data,
    "species" => species,
))

## Intra-genus distance lower bounds

In [69]:
genus_pw_mindists = zeros(Float32, TriSymmetric(), ngenera);

In [83]:
for i in 1:ngenera
    idxs1 = searchsorted(species_to_genus, i)
    for j in 1:(i-1)
        idxs2 = searchsorted(species_to_genus, j)
        genus_pw_mindists[i, j] = minimum(species_pw_mindists[idxs1, idxs2])
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [88]:
save(outfiles[:genus_min_dists], Dict(
    "data" => genus_pw_mindists.data,
    "genera" => genera,
))